In [1]:
import pandas as pd

df = pd.read_csv('product_reviews_dirty.csv')
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40593 entries, 0 to 40606
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    40593 non-null  int64 
 1   text          40593 non-null  object
 2   rating        40593 non-null  int64 
 3   category      40593 non-null  object
 4   product_name  40593 non-null  object
 5   product_id    40593 non-null  int64 
 6   sold          40593 non-null  object
 7   shop_id       40593 non-null  int64 
 8   product_url   40593 non-null  object
dtypes: int64(4), object(5)
memory usage: 3.1+ MB


In [16]:
# pick the first 5000 rows
df_5000 = df.head(5000)

In [14]:
import preprocessing as pre
import pickle
import numpy as np
with open('Model/TFIDF Model - 80.pkl', 'rb') as f:
    tfidf_model_80 = pickle.load(f)

with open('Model/NeuralNetwork_model_80', 'rb') as f:
    nn_model_80 = pickle.load(f)

def just_prediction(text):
    tfidf_model = tfidf_model_80
    nn_model = nn_model_80

    text_clean = pre.preprocess(text)
    text_tfidf_vector = tfidf_model.transform(text_clean)
    text_tfidf_vector_reshape = text_tfidf_vector.reshape(-1, 1)
    _, _, _, _, nn_A2 = nn_model.forward(text_tfidf_vector_reshape)

    prediction = np.argmax(nn_A2).item()
    map_label = {0: 'Product', 1: 'Customer Service', 2: 'Shipping/Delivery'}
    result = map_label[prediction]
    return result

In [19]:
df['class_prediction'] = df['text'].apply(lambda x: just_prediction(x))
df.head()

,Unnamed: 0,text,rating,category,product_name,product_id,sold,shop_id,product_url,class_prediction
0,1,Barang sesuai pesanan dan cepat sampai,5,pertukangan,Staples Dekorasi Staples Kayu + Refill 8mm - S...,418660637,1,1740837,https://www.tokopedia.com/shakaonline87/staple...,Customer Service
1,2,Barang bagus harga murah,5,pertukangan,STAPLE GUN ATS 3 WAY TACKER - STAPLES JOK TEMB...,416032545,11,1477109,https://www.tokopedia.com/juraganperkakas/stap...,Product
2,3,Paket rapi...mantap....cepat....sampe ke tujuan,5,pertukangan,STAPLE GUN ATS 3 WAY TACKER - STAPLES JOK TEMB...,416032545,11,1477109,https://www.tokopedia.com/juraganperkakas/stap...,Shipping/Delivery
3,4,ya saya puas dgn barangnya,5,pertukangan,ALAT STAPLES TEMBAK &#40;AIR NAILER GUN&#41; O...,102279869,5,771395,https://www.tokopedia.com/kamarmesin/alat-stap...,Customer Service
4,5,Responya luar biasa b mantap,5,pertukangan,Isi Refill Staples Jok Kulit Motor / Staple Gu...,190679689,787,969999,https://www.tokopedia.com/mitrapersada/isi-ref...,Shipping/Delivery


In [20]:
df.to_csv('product_reviews_dirty_with_prediction.csv', index=False)